In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00001-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00002-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/README.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model.bin.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/special_tokens_map.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/.gitattributes
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.model
/kaggle/input/llama-2/pyt

In [2]:
!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.33.1
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

  Obtaining dependency information for transformers==4.33.1 from https://files.pythonhosted.org/packages/13/30/54b59e73400df3de506ad8630284e9fd63f4b94f735423d55fc342181037/transformers-4.33.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers!=0.11.3,<0.14,>=0.11.1 from https://files.pythonhosted.org/packages/94/60/ff26cce378023624ffcad91edaa4871f561d6ba7295185c45037ddba80e2/tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstallin

In [3]:
from torch import cuda
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers

model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Bangla RQA dataset Loading

In [5]:
from datasets import load_dataset

df_train = load_dataset("sartajekram/BanglaRQA", split="train")
df_validation = load_dataset("sartajekram/BanglaRQA", split="validation")
df_test = load_dataset("sartajekram/BanglaRQA", split="test")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset bangla_rqa downloaded and prepared to /root/.cache/huggingface/datasets/sartajekram___bangla_rqa/BanglaRQA/0.0.1/d16b566a8f65c0812aa2ca603b7833c4eb3c1476621896496af7f94d7743ec7c. Subsequent calls will reuse this data.


In [6]:
# Import Pandas for convenient table display
import pandas as pd

# Create a DataFrame from the first few examples
BanglaRQA_train = pd.DataFrame(df_train)
BanglaRQA_validation = pd.DataFrame(df_validation)
BanglaRQA_test = pd.DataFrame(df_test)

# Display the DataFrame
display(BanglaRQA_train)

,passage_id,title,context,question_id,question_text,is_answerable,question_type,answers
0,bn_wiki_2977,ফাজিল পরীক্ষা,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,bn_wiki_2977_01,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,1,confirmation,"{'answer_text': ['হ্যাঁ', 'হ্যাঁ '], 'answer_t..."
1,bn_wiki_2977,ফাজিল পরীক্ষা,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,bn_wiki_2977_02,বাংলাদেশের ফাজিল পরীক্ষা ইসলামি আরবি বিশ্ববিদ...,0,causal,"{'answer_text': ['', ''], 'answer_type': ['', ..."
2,bn_wiki_2977,ফাজিল পরীক্ষা,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,bn_wiki_2977_03,কত সালে ঢাকা আলিয়া মাদ্রাসা ঢাকায় স্থানান্তর...,1,factoid,"{'answer_text': ['১৯৪৭', '১৯৪৭'], 'answer_type..."
3,bn_wiki_2977,ফাজিল পরীক্ষা,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,bn_wiki_2977_04,মাদ্রাসা-ই-আলিয়া ঢাকায় স্থানান্তরিত হলে কত স...,1,factoid,"{'answer_text': ['১৯৪৮', '১৯৪৮'], 'answer_type..."
4,bn_wiki_2977,ফাজিল পরীক্ষা,ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাস...,bn_wiki_2977_05,মোট কত বছর ব্যাপী আলিয়া মাদ্রাসার পূর্ণাঙ্গ আ...,1,factoid,"{'answer_text': ['১৬', '১৬'], 'answer_type': [..."
...,...,...,...,...,...,...,...,...
11907,bn_wiki_2528,নারী ক্ষমতায়ন,বিংশ শতকের শেষ ভাগে ওয়েব ক্রমবর্ধমান অ্যাক্সে...,bn_wiki_2528_01,ইন্টারনেটের ব্যবহার নারীদেরকে ক্ষমতায়িত করার স...,1,factoid,"{'answer_text': ['বিংশ শতকের শেষ ভাগে', 'বিংশ ..."
11908,bn_wiki_2528,নারী ক্ষমতায়ন,বিংশ শতকের শেষ ভাগে ওয়েব ক্রমবর্ধমান অ্যাক্সে...,bn_wiki_2528_02,ব্লগিং কি নারী শিক্ষার ক্ষমতায়নে অন্যতম একটি স...,1,confirmation,"{'answer_text': ['হ্যাঁ', 'হ্যাঁ '], 'answer_t..."
11909,bn_wiki_2528,নারী ক্ষমতায়ন,বিংশ শতকের শেষ ভাগে ওয়েব ক্রমবর্ধমান অ্যাক্সে...,bn_wiki_2528_03,ই-লার্নিং এর সাহায্যে নারীরা অগ্রণী বিশ্বায়নে ...,1,causal,{'answer_text': ['শিক্ষাগতভাবে নিজেদের ক্ষমতায...
11910,bn_wiki_2528,নারী ক্ষমতায়ন,বিংশ শতকের শেষ ভাগে ওয়েব ক্রমবর্ধমান অ্যাক্সে...,bn_wiki_2528_04,এস ডি জি কী?,0,factoid,"{'answer_text': ['', ''], 'answer_type': ['', ..."


In [7]:
BanglaRQA_train.columns

Index(['passage_id', 'title', 'context', 'question_id', 'question_text',
       'is_answerable', 'question_type', 'answers'],
      dtype='object')

### Training Data Preprocessing

In [8]:
pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-0vgapto6
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-0vgapto6
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=9571523e50df1457c3e68af124bb0c6be42055c2c259c8eda0c55ae840ad5a59
  Stored in directory: /tmp/pip-ephem-wheel-cache-fpezbnqg/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=73d00a389c65f925500893840cd5b653

In [9]:
BanglaRQA_train.dtypes

passage_id       object
title            object
context          object
question_id      object
question_text    object
is_answerable    object
question_type    object
answers          object
dtype: object

In [10]:
#BanglaRQA_train.loc[0, 'context']
len(BanglaRQA_train)

11912

In [11]:
from normalizer import normalize
import string, re
def remove_punc(text):
  exclude = set(string.punctuation)
  exclude.remove('?')
  return "".join(ch for ch in text if ch not in exclude)

context = []
question = []
answer = []

for i in range(len(BanglaRQA_train)):
        context.append(remove_punc(normalize(BanglaRQA_train.loc[i, 'context'])))
        question.append(remove_punc(normalize(BanglaRQA_train.loc[i, 'question_text'])))
        answer.append(normalize(BanglaRQA_train.loc[i, 'answers']['answer_text'][0]))
     

In [12]:
print(question[0])
print(context[0])
print(len(context))
print(len(question))
print(len(answer))

ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাসায় অনুষ্ঠিত একটি সরকারি পরীক্ষা ?
ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাসায় অনুষ্ঠিত একটি সরকারি পরীক্ষা। ফাজিল পরীক্ষা বাংলাদেশে ডিগ্রি সমমানের কখনো স্নাতক সমমানের একটি পরীক্ষা যা একটি ফাজিল মাদ্রাসায় অনুষ্ঠিত হয়ে থাকে। তবে ভারতে ফাজিল পরীক্ষাকে উচ্চ মাধ্যমিক শ্রেণীর ১১ বা ১২ ক্লাস মান বলে বিবেচিত করা হয়। ফাজিল পরীক্ষা বাংলাদেশ ভারত ও পাকিস্তানের সরকারি স্বীকৃত আলিয়া মাদরাসায় প্রচলিত রয়েছে। বাংলাদেশের ফাজিল পরীক্ষা ইসলামি আরবি বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়ে থাকে ও ভারতের ফাজিল পরীক্ষা পশ্চিমবঙ্গ মাদ্রাসা শিক্ষা পর্ষদের অধীনে অনুষ্ঠিত হয়ে থাকে। ১৯৪৭ সালে ঢাকা আলিয়া মাদ্রাসা ঢাকায় স্থানান্তরের পূর্বে বাংলাদেশ ও ভারতের ফাজিল পরীক্ষা কলকাতা আলিয়া মাদ্রাসার অধীনে অনুষ্ঠিত হতো। ফাযিল পরীক্ষা বর্তমানে ইসলামি আরবী বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়। যা পূর্বে মাদরাসা বোর্ড ও ইসলামি বিশ্ববিদ্যালয়ের আধীনে অনুষ্ঠিত হত। মাদ্রাসাইআলিয়া ঢাকায় স্থানান্তরিত হলে ১৯৪৮ সালে মাদ্রাসা বোর্ডের ফাজিলগুলো পরীক্ষা ঢাকা বিশ্ববিদ্যালয় ক

In [13]:
# for i in range(len(context)):
#   if question[i].find('?') == -1:
#     print(f"Context: {context[i]}")
#     print(f"Question: {question[i]}")
#     print("---")  # Separator between rows


In [14]:
for i in range(len(question)):
    if question[i].endswith('|'):
        question[i] = question[i][:-1] + '?'  # Remove '|' and add '?'


In [15]:
for i in range(len(question)):
    if question[i].endswith(''):
        question[i] = question[i][:-1] + '?'  # Remove '' and add '?'


In [16]:
for i in range(len(context)):
  context[i] = context[i].replace('।', ' ।')
  answer[i] = answer[i].replace('।', ' ।')
  question[i] = question[i].replace('।', ' ।')

print(context[0])

ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাসায় অনুষ্ঠিত একটি সরকারি পরীক্ষা । ফাজিল পরীক্ষা বাংলাদেশে ডিগ্রি সমমানের কখনো স্নাতক সমমানের একটি পরীক্ষা যা একটি ফাজিল মাদ্রাসায় অনুষ্ঠিত হয়ে থাকে । তবে ভারতে ফাজিল পরীক্ষাকে উচ্চ মাধ্যমিক শ্রেণীর ১১ বা ১২ ক্লাস মান বলে বিবেচিত করা হয় । ফাজিল পরীক্ষা বাংলাদেশ ভারত ও পাকিস্তানের সরকারি স্বীকৃত আলিয়া মাদরাসায় প্রচলিত রয়েছে । বাংলাদেশের ফাজিল পরীক্ষা ইসলামি আরবি বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়ে থাকে ও ভারতের ফাজিল পরীক্ষা পশ্চিমবঙ্গ মাদ্রাসা শিক্ষা পর্ষদের অধীনে অনুষ্ঠিত হয়ে থাকে । ১৯৪৭ সালে ঢাকা আলিয়া মাদ্রাসা ঢাকায় স্থানান্তরের পূর্বে বাংলাদেশ ও ভারতের ফাজিল পরীক্ষা কলকাতা আলিয়া মাদ্রাসার অধীনে অনুষ্ঠিত হতো । ফাযিল পরীক্ষা বর্তমানে ইসলামি আরবী বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয় । যা পূর্বে মাদরাসা বোর্ড ও ইসলামি বিশ্ববিদ্যালয়ের আধীনে অনুষ্ঠিত হত । মাদ্রাসাইআলিয়া ঢাকায় স্থানান্তরিত হলে ১৯৪৮ সালে মাদ্রাসা বোর্ডের ফাজিলগুলো পরীক্ষা ঢাকা বিশ্ববিদ্যালয় কর্তৃক গৃহীত হতো । ১৯৭৫ সালের কুদরতএখুদা শিক্ষা কমিশনের সুপারিশে মাদ্রাসা 

In [17]:
for i in range(len(context)):
  if question[i].find(' ?') == -1:
    question[i] = question[i].replace('?', ' ?')

In [18]:
c = 0
for i in range(len(context)):
  if question[i].find(' ?') != -1:
    c = c+1
print(c)

11912


In [19]:
question_context = []

for i in range(len(context)):
  context[i] = 'হ্যাঁ না । ' + context[i]
  question_context.append(question[i] + ' ' + context[i])
  a = question[i] + ' ' + context[i]
  ans_spans = answer[i].split('; ')

  for ans in ans_spans:
    ans = remove_punc(ans)
    if(ans == ''):
      continue
    else:
      ansx = ans.split()
      x = ''
      for j in range(len(ansx)):
        if(j == 0):
          x = 'B'
        else:
          x = x + ' I'
      a = a.replace(ans, x, 1)

  a = a.split()
  for j, x in enumerate(a):
    if(x != 'B' and x!= 'I'):
      a[j] = 'O'

  answer[i] = a


In [20]:
# print(ans_spans)

In [21]:
print(question_context[565])
print(answer[565])

ডরিক গ্রীক কোন অঞ্চলের আঞ্চলিক ভাষা ছিল ? হ্যাঁ না । আর্কিমিডিস তার কাজের লিখিত রূপের জন্য ডরিক গ্রিক ভাষা ব্যবহার করতেন যা প্রাচীন সিরাকিউজের আঞ্চলিক ভাষা হিসেবে প্রচলিত ছিল । আর্কিমিডিসের অধিকাংশ কাজ ইউক্লিডের কাজের মত সংরক্ষিত হয়নি তার সাতটি থীসিসের কথা জানা যায় কেবলমাত্র অন্যদের কাজের রেফারেন্স থেকে । পাপ্পাস অভ আলেকজান্দ্রিয়া আর্কিমিডিসের অন স্ফীয়ার মেকিং এবং বহুতল বিশিষ্ট বস্তুর উপর আরএকটি কাজের কথা উল্লেখ করেছেন । অপরদিকে থেরন অভ আলেকজান্দ্রিয়া প্রতিসরণ সম্পর্কে আর্কিমিডিসের হারিয়ে যাওয়া একটি লেখনী ক্যাটোপট্রিকা এর উল্লেখ করেন । জীবদ্দশায় আর্কিমিডিস তার কাজের প্রচারের জন্য আলেকজান্দ্রিয়ার গণিতবিদদের উপর নির্ভর করতেন । বাইজান্টাইন স্থপতি ইসিডোর অভ মিলেতাস আর্কিমিডিসের লেখনীগুলোকে একত্রিত করেন পরবর্তীতে ষষ্ঠ শতকে ইউতোশিয়াস অভ আসকালোন তার কাজের উপর লিখিত বিবরণ প্রকাশ করার পর আর্কিমিডিসের কাজ বৃহত্তর জনগোষ্ঠীর কাছে পরিচিত হয়ে ওঠে । আর্কিমিডিসের কাজ থাবিত ইবনে কুররা ৮৩৬৯০১ খ্রিষ্টাব্দ আরবিতে এবং জেরার্ড অভ ক্রেমোনা ১১৪৭১১৮৭ খ্রিষ্টাব্দ ল্যাটিনে অনুবাদ করেন । রেনেসাঁর সময় 

In [22]:
labels_to_ids = {k: v for v, k in enumerate(['B', 'I', 'O'])}
ids_to_labels = {v: k for v, k in enumerate(['B', 'I', 'O'])}
print(labels_to_ids)
print(ids_to_labels)

{'B': 0, 'I': 1, 'O': 2}
{0: 'B', 1: 'I', 2: 'O'}


### Validation Data Preprocessing

In [23]:
len(BanglaRQA_validation)

1484

In [24]:
from normalizer import normalize
import string, re
def remove_punc(text):
  exclude = set(string.punctuation)
  exclude.remove('?')
  return "".join(ch for ch in text if ch not in exclude)

context_val = []
question_val = []
answer_val = []

for i in range(len(BanglaRQA_validation)):
        context_val.append(remove_punc(normalize(BanglaRQA_validation.loc[i, 'context'])))
        question_val.append(remove_punc(normalize(BanglaRQA_validation.loc[i, 'question_text'])))
        answer_val.append(normalize(BanglaRQA_validation.loc[i, 'answers']['answer_text'][0]))
     

In [25]:
print(len(question_val))
print(context_val[0])

1484
আস্যাইরিয়ান ও ব্যাবলিয়ান সাহিত্যে সরগন তার অবনমিত অবস্থান থেকে ক্ষমতায় উত্থান ও মেসোপটেমিয়া অভিযানের জন্য কিংবদন্তি গল্পের মুল উপজীব্যে পরিনত হয়েছিলেন এরকম আরও কিছু কিছু আংশিক কিংবদন্তী উপাখ্যান ছাড়াও সারগনের খোদ নিজের অনেক লিপি আছে যদিও তার বেশিরভাগ পরবর্তী সংস্করনগুলি থেকে নেওয়া। ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফলক এর অংশবিশেষ আছে যা সুসা যেখানে এগুলি খুব সম্ভবত মেসোপটেমিয়া ত্থেকে দ্বাদশ শতাব্দীতে স্থানান্তরিত করা হয়েছিল থেকে পুনঃউদ্ধার করা হয়েছিল। দৃশ্যত সরগন সেমেটিক আক্কাদীয়ান ভাষার লিপির লিখিত আকারে প্রসার ঘটিয়েছিলেন তিনি আক্কাদ শহর প্রতিষ্ঠিত করার প্রথমদিকে নিজেকে প্রায়শয়ই আক্কাদীয়ান রাজা হিসিবে প্রচার করতেন পরে তিনি কোন এক সময় কিস শহর অধিগ্রহণ করে নেন পরবর্তীতে মেসোপটেমিয়ার বৃহদাংশও দখল করে নেন এবং ক্রমে ক্রমে সরগন আক্কদীয়ান রাজা ইনান্নার তত্ত্বাবধায়ক কিস এর রাজা আনুর স্থলাভিষিক্ত রাজ্যেরমেসোপটেমিয়া রাজা এনলিলের রাজ্যপালএনসি নামে নিজেকে প্রচার করে ছিলেন। যদিও সুমেরিয়ান রাজাদের তালিকার অনেক অনুলিপিতে সারগনের শাসনকাল ৫৬ ৫৫ বা ৫৪ বছর বলা হয়েছে কিন্তু তা

In [26]:
for i in range(len(question_val)):
    if question_val[i].endswith('|'):
        question_val[i] = question_val[i][:-1] + '?'  # Remove '|' and add '?'


In [27]:
for i in range(len(question_val)):
    if question_val[i].endswith(''):
        question_val[i] = question_val[i][:-1] + '?'  # Remove '' and add '?'


In [28]:
for i in range(len(context_val)):
  context_val[i] = context_val[i].replace('।', ' ।')
  answer_val[i] = answer_val[i].replace('।', ' ।')
  question_val[i] = question_val[i].replace('।', ' ।')

print(context[0])

হ্যাঁ না । ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাসায় অনুষ্ঠিত একটি সরকারি পরীক্ষা । ফাজিল পরীক্ষা বাংলাদেশে ডিগ্রি সমমানের কখনো স্নাতক সমমানের একটি পরীক্ষা যা একটি ফাজিল মাদ্রাসায় অনুষ্ঠিত হয়ে থাকে । তবে ভারতে ফাজিল পরীক্ষাকে উচ্চ মাধ্যমিক শ্রেণীর ১১ বা ১২ ক্লাস মান বলে বিবেচিত করা হয় । ফাজিল পরীক্ষা বাংলাদেশ ভারত ও পাকিস্তানের সরকারি স্বীকৃত আলিয়া মাদরাসায় প্রচলিত রয়েছে । বাংলাদেশের ফাজিল পরীক্ষা ইসলামি আরবি বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়ে থাকে ও ভারতের ফাজিল পরীক্ষা পশ্চিমবঙ্গ মাদ্রাসা শিক্ষা পর্ষদের অধীনে অনুষ্ঠিত হয়ে থাকে । ১৯৪৭ সালে ঢাকা আলিয়া মাদ্রাসা ঢাকায় স্থানান্তরের পূর্বে বাংলাদেশ ও ভারতের ফাজিল পরীক্ষা কলকাতা আলিয়া মাদ্রাসার অধীনে অনুষ্ঠিত হতো । ফাযিল পরীক্ষা বর্তমানে ইসলামি আরবী বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয় । যা পূর্বে মাদরাসা বোর্ড ও ইসলামি বিশ্ববিদ্যালয়ের আধীনে অনুষ্ঠিত হত । মাদ্রাসাইআলিয়া ঢাকায় স্থানান্তরিত হলে ১৯৪৮ সালে মাদ্রাসা বোর্ডের ফাজিলগুলো পরীক্ষা ঢাকা বিশ্ববিদ্যালয় কর্তৃক গৃহীত হতো । ১৯৭৫ সালের কুদরতএখুদা শিক্ষা কমিশনের সুপারিশ

In [29]:
for i in range(len(context_val)):
  if question_val[i].find(' ?') == -1:
    question_val[i] = question_val[i].replace('?', ' ?')

In [30]:
c = 0
for i in range(len(context_val)):
  if question_val[i].find(' ?') != -1:
    c = c+1
print(c)

1484


In [31]:
print(question_val[0])

কোন জাদুঘরে সারগনিক বিজয় ফলক আছে ?


In [32]:
question_context_val = []

for i in range(len(context_val)):
  context_val[i] = 'হ্যাঁ না । ' + context_val[i]
  question_context_val.append(question_val[i] + ' ' + context_val[i])
  a = question_val[i] + ' ' + context_val[i]
  ans_spans = answer_val[i].split('; ')

  for ans in ans_spans:
    ans = remove_punc(ans)
    if(ans == ''):
      continue
    else:
      ansx = ans.split()
      x = ''
      for j in range(len(ansx)):
        if(j == 0):
          x = 'B'
        else:
          x = x + ' I'
      a = a.replace(ans, x, 1)

  a = a.split()
  for j, x in enumerate(a):
    if(x != 'B' and x!= 'I'):
      a[j] = 'O'

  answer_val[i] = a


In [33]:
print(question_context_val[565])
print(answer_val[565])

আরপানেট এর জন্ম হয় কত সালে ? হ্যাঁ না । বিভিন্ন রকম ইংরেজি বানান রয়েছে বাছাই করার যেগুলো একে অন্যের সাথে মিলে না । ইমেইল বানানটি আইইটিএফ রিকোয়েস্ট ফর কমেন্ট এবং তাদের কাজের দল এবং বিস্তৃত ভাবে স্টাইল গাইডস দ্বারা ব্যবহৃত হয় । এই বানানটি বিভিন্ন অভিধানেও দেখা যায় । ইমেইল এই বানানটি কিছু প্রথম সারির খ্যাতিমান সাংবাদিক এবং বিভিন্ন টেকনিকাল স্টাইল গাইড দ্বারা সুপারিশ করা হয়েছিল । করপাস অব কনটেমপরারি আমেরিকান ইংলিশ তথ্য অনুযায়ী এই বানানটি আমেরিকান সম্পাদনাতে পুনপুন প্রকাশিত হয়েছিল । মেইল এই বানান প্রধান আরএফসি ব্যবহার করত । সেবা মেইল হিসেবে বুঝানো হত এবং একটি ইলেক্ট্রনিক মেইলকে ম্যাসেজ বলা হত । ইমেইল প্রথম এম অক্ষরটি বড় হাতের হরফে লেখা হত এটা অর্পানেট ব্যবহারকারী এবং প্রথম দিকের উন্নয়নকারী ইউনিক্স সিএমএস এ্যপললিংক ইওয়ার্ল্ড এওএল জিনি এবং হটমেইল কর্তৃক ব্যবহৃত হত । ইমেইল এই বানানটি আরএফসির প্রথাগত বানান যেখানে এটি ব্যবহৃত হত অথর বা প্রবর্তকের ঠিকানা হিসেবে । ইমেইল প্রথম ই অক্ষরটি কে বড় হরফে লেখার যেটা এক্সরে বা টিশার্ট এর মতো । বিশ শতকের ষাটসত্তরের দশকে ইন্টারনেট প্রোটোকলএর মাধ্য

### Further Processing

In [34]:
from collections import Counter

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    common_tokens = sum(common_tokens.values())
    
    # if there are no common tokens then f1 = 0
    if common_tokens == 0:
        return 0
    
    prec = 1.0 * common_tokens / len(pred_tokens)
    rec = 1.0 * common_tokens / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [35]:
label_all_tokens = False

def align_label(qc, labels):
    #print(qc)
    #print(labels)
    
    tokenized_inputs = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
    
    word_ids = tokenized_inputs.word_ids()
    print(word_ids)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        #print(word_idx)
        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        
        #print(label_ids)
        previous_word_idx = word_idx

    return label_ids

In [36]:
#question_context[0]


In [37]:
# answer[0]

### Not needed From Here

From Here it is commented.

In [38]:
# from transformers import AutoModelForTokenClassification, AutoTokenizer
# from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
# import torch

# model = AutoModelForTokenClassification.from_pretrained("csebuetnlp/banglabert", num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert", use_fast=True,)

In [39]:
# ins = []
# outs = []
# for i in range(3):
#   ins.append(tokenizer(question_context[i], max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt"))
#   outs.append(align_label(question_context[i], answer[i]))

In [40]:
# ins[2]

In [41]:
# outs[2]

Till now

In [42]:
# from torch.utils.data import Dataset

# class CustomDataset(Dataset):
#     def __init__(self, data_dict):
#         self.data = data_dict

#     def __len__(self):
#         return len(self.data["inputs"])  # Assuming equal lengths for inputs and outputs

# def __getitem__(self, idx):
#     inputs, outputs = super().__getitem__(idx)  # Assuming `inputs` is a BatchEncoding
#     return {"input_ids": inputs, "outputs": outputs}  # Wrap in a dictionary


# training_dataset = CustomDataset(training_data)  # Pass the dictionary to create the dataset

In [43]:
# print(type(ins[0]))

In [44]:
# print(type(outs[0]))

In [45]:
# def combine_to_dict(source_data, target_data):
    
# #    combined_data = [{"qc": src.strip(), "label": tgt.strip()} for src, tgt in zip(source_data, target_data)]
#     return Dataset.from_dict({
#          "qc": source_data,
#          "label": target_data
#      }
#     )

# # Load train, and validation data into separate variables
# train_data = combine_to_dict(ins, outs)
# validation_data = combine_to_dict(ins_val, outs_val)

In [46]:
# training_data = []  # Store preprocessed data as dictionaries

# for i in range(len(context)):
#     # Tokenize text input
#     tokenized_text = tokenizer(question_context[i], max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

#     # Align label (assuming `align_label` already aligns with tokenized input)
#     aligned_label = align_label(question_context[i], answer[i])

#     # Create dictionary for this example
#     data_dict = {
#         "text": tokenized_text,
#         "label": aligned_label
#     }

#     training_data.append(data_dict)

In [47]:
# training_data

In [48]:
# validation_data = []  # Store preprocessed data as dictionaries

# for i in range(len(context_val)):
#     # Tokenize text input
#     tokenized_text = tokenizer(question_context_val[i], max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

#     # Align label (assuming `align_label` already aligns with tokenized input)
#     aligned_label = align_label(question_context_val[i], answer_val[i])

#     # Create dictionary for this example
#     data_dict_val = {
#         "text": tokenized_text,
#         "label": aligned_label
#     }

#     validation_data.append(data_dict_val)

In [49]:
# print(type(validation_data))

In [50]:
# class DataSequence(torch.utils.data.Dataset):
   
#     def __init__(self, i, o, tokenizer):
#         self.texts = i
#         self.labels = o
#         self.tokenizer = tokenizer  # Store the tokenizer

#     def __len__(self):

#         return len(self.labels)

#     def get_batch_data(self, idx):

#         return self.texts[idx]

#     def get_batch_labels(self, idx):

#         return torch.LongTensor(self.labels[idx])

#     def __getitem__(self, idx):
#         text_data = self.get_batch_data(idx)
#         input_ids = self.tokenizer(text_data, return_tensors="pt").input_ids  # Add tokenization
#         batch_labels = self.get_batch_labels(idx)
#         return {"input_ids": input_ids, "labels": batch_labels}  # Return as a dictionary



In [51]:
# train_dataset = DataSequence(ins, outs, tokenizer)

In [52]:
# print(type(train_dataset))

### Validation

In [53]:
# ins_val = []
# outs_val = []
# for i in range(len(context_val)):
#   ins_val.append(tokenizer(question_context_val[i], max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt"))
#   outs_val.append(align_label(question_context_val[i], answer_val[i]))

In [54]:
# validation_dataset = DataSequence(ins_val, outs_val)

In [55]:
# print(type(train_dataset))

In [56]:
# def align_word_ids(qc):
  
#     tokenized_inputs = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
#     #print(qc)
#     word_ids = tokenized_inputs.word_ids()
#     #print(word_ids)
#     previous_word_idx = None
#     label_ids = []

#     for word_idx in word_ids:

#         if word_idx is None:
#             label_ids.append(-100)

#         elif word_idx != previous_word_idx:
#             try:
#                 label_ids.append(1)
#             except:
#                 label_ids.append(-100)
#         else:
#             try:
#                 label_ids.append(1 if label_all_tokens else -100)
#             except:
#                 label_ids.append(-100)
#         previous_word_idx = word_idx

#     return label_ids

# def evaluate_one_text(qc):

#     text = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
    
#     mask = text['attention_mask'].to(device)
#     input_id = text['input_ids'].to(device)
#     label_ids = torch.Tensor(align_word_ids(qc)).unsqueeze(0).to(device)

#     logits = model(input_id, mask, None)
#     #print(logits[0])
#     #print(len(logits[0][0]))
#     #logits_clean = logits[0][label_ids != -100]
#     #print(logits_clean)
#     #print(logits[0][0])

#     predictions = logits[0][0].argmax(dim=1).tolist()
#     #print(predictions)
#     prediction_label = [ids_to_labels[i] for i in predictions]
#     #print(prediction_label)

#     #print(input_id)
#     a = tokenizer.convert_ids_to_tokens(input_id[0])
#     #print(a)
#     '''
#     print(prediction_label)
#     print(len(prediction_label))
#     print(a)
#     print(len(a))
#     '''
#     ans = ''
#     ind = True
#     for i in range(len(prediction_label)):
#       #print(i)
#       if(prediction_label[i] == 'B' and ind == True):
#         ans = ans + a[i]
#         ind = False
#       elif (prediction_label[i] == 'B'):
#         ans = ans + '; ' + a[i]
#       elif (label_ids[0][i] == -100 and prediction_label[i] == 'I'):
#         ans = ans + a[i]
#       elif (prediction_label[i] == 'I'):
#         ans = ans + ' ' + a[i]

#     return ans
     


In [57]:
# checkpoint_dir = '/kaggle/working/' 

In [58]:
# from torch.utils.data import DataLoader
# from transformers import AdamW
# from tqdm import tqdm

# # setup GPU/CPU
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# # move model over to detected device
# model.to(device)

# model.config.architectures = ['ElectraForTokenClassification']
# # activate training mode of model
# model.train()
# # initialize adam optimizer with weight decay (reduces chance of overfitting)
# optim = AdamW(model.parameters(), lr=2e-5)

# # initialize data loader for training data
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# for epoch in range(15):
#     # set model to train mode
#     model.train()
#     model_name = 'banglaBERT_model_weights_epoch_' + str(epoch) + '.pth'
#     # setup loop (we use tqdm for the progress bar)
#     loop = tqdm(train_loader, leave=True)
#     for train_data, train_label in loop:
#         # initialize calculated gradients (from prev step)
#         optim.zero_grad()
#         # pull all the tensor batches required for training
#         input_ids = train_data['input_ids'].squeeze(1).to(device)
#         attention_mask = train_data['attention_mask'].squeeze(1).to(device)
#         labels = train_label.to(device)
      
#         outputs = model(input_ids=input_ids, 
#                           attention_mask=attention_mask, 
#                             labels = labels)
#         #print(outputs)
#         # extract loss
#         loss = outputs[0]
#         #print(loss)
#         # calculate loss for every parameter that needs grad update 
#         loss.backward()
#         # update parameters
#         optim.step()

#         #print(outputs)
#         #print(model.config.architectures)
#         # print relevant info to progress bar
#         loop.set_description(f'Epoch {epoch}')
#         loop.set_postfix(loss=loss.item())
#         torch.save({
#         'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optim.state_dict(),
#         'loss': loss,
        
#         }, os.path.join(checkpoint_dir,model_name))
    
#     optim.zero_grad()
#     model.eval()


#     f1_total = 0.0
#     em_total = 0.0

#     with torch.no_grad():
#       l = len(context_val)
#       for i in range(l):
#           pred = evaluate_one_text(question_val[i] + ' হ্যাঁ না । ' + context_val[i])
#           pred = pred.replace('#', '')
#           #if(i<10):
#             #print(pred)
#           f1_total = f1_total + compute_f1(pred, answer_val[i])
#           em_total = em_total + compute_exact_match(pred, answer_val[i])

#       print("Epoch", epoch)
#       print('F1: ', f1_total/l)
#       print('EM: ', em_total/l)  

In [59]:
# with torch.no_grad():
#       l = len(context_val)
#       for i in range(l):
#           print(i)
#           pred = evaluate_one_text(question_context[i])
#           pred = pred.replace('#', '')
#           if(i<10):
#             print(pred)
#             print(answer_raw[i])
#           else:
#             break

### Model Load

From there, this is commented

In [60]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# import transformers

# model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model_2 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# tokenizer_2 = AutoTokenizer.from_pretrained(model_id)
# tokenizer_2.pad_token = tokenizer.eos_token
# tokenizer_2.padding_side = "right"

In [61]:
# def print_trainable_parameters(model):
#     """
#     Prints the number of trainable parameters in the model.
#     """
#     trainable_params = 0
#     all_param = 0
#     for _, param in model.named_parameters():

#         all_param += param.numel()
#         if param.requires_grad:
#             trainable_params += param.numel()
#     print(
#         f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
#     )

In [62]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

# print(model)

In [63]:
# from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=64,
#     # target_modules=["query_key_value"],
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model_2, lora_config)
# print_trainable_parameters(model_2)

In [64]:
# import tensorflow as tf
# import os

# log_dir = '/kaggle/working/llama2-docsum-adapter/runs'  
# summary_writer = tf.summary.create_file_writer(log_dir)

# # with summary_writer.as_default():
# #     tf.summary.scalar('loss', loss, step=epoch)

# os.makedirs(log_dir, exist_ok=True)



# OUTPUT_DIR = "/kaggle/working/llama2-docsum-adapter"

# # %load_ext tensorboard
# # %tensorboard --logdir llama2-docsum-adapter/runs

In [65]:
# from transformers import TrainingArguments

# training_arguments = TrainingArguments(
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     optim="paged_adamw_32bit",
#     logging_steps=1,
#     learning_rate=1e-4,
#     fp16=True,
#     max_grad_norm=0.3,
#     num_train_epochs=3,
#     evaluation_strategy="steps",
#     eval_steps=0.2,
#     warmup_ratio=0.05,
#     save_strategy="epoch",
#     group_by_length=True,
#     output_dir=OUTPUT_DIR,
# #     report_to="tensorboard",
#     save_safetensors=True,
#     lr_scheduler_type="cosine",
#     seed=42,
# )
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [66]:
# from trl import SFTTrainer
# trainer = SFTTrainer(
#     model=model_2,
#     train_dataset=training_dataset,
# #    eval_dataset=validation_data,
#     peft_config=lora_config,
#     dataset_text_field="text",
#     max_seq_length=4096,
#     tokenizer=tokenizer,
#     args=training_arguments,
# )

# trainer.train()